In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline
sns.set()

# Load data

In [7]:
!kaggle competitions download -c test-recsysb

test-recsys.zip: Skipping, found more recently modified local copy (use --force to force download)


In [111]:
categories = pd.read_csv('./test-recsys/kaggle_tab_1345/tab_3_categories.csv')
categories.set_index('id', inplace=True)
categories.dtypes

name         object
parent_id     int64
dtype: object

In [112]:
import glob

path = r'.\test-recsys\sbermarket_tab_2_4'
all_files = glob.glob(path + "/*.csv")

products = pd.concat((pd.read_csv(f) for f in all_files))

In [113]:
L = 4

In [114]:
categories_map = categories['parent_id']
products_e = products
products_e['category_0'] = products_e.master_category_id
print(f"level 0 ready")
for category_level in range(1, L):
    current_category_column = f'category_{category_level - 1}'
    next_category_column = f'category_{category_level}'
    products_e[next_category_column] = products_e[current_category_column].map(categories_map)
    print(f"level {category_level} ready")

level 0 ready
level 1 ready
level 2 ready
level 3 ready


In [117]:
products_e[['product_id', 'category_0', 'category_1', 'category_2', 'category_3']].head(10)

,product_id,category_0,category_1,category_2,category_3
0,5061192,177.0,175.0,174.0,0.0
1,64119,177.0,175.0,174.0,0.0
2,54728,85.0,84.0,1.0,0.0
3,655,85.0,84.0,1.0,0.0
4,63057,85.0,84.0,1.0,0.0
5,100345,85.0,84.0,1.0,0.0
6,4831055,85.0,84.0,1.0,0.0
7,709,91.0,90.0,1.0,0.0
8,4277030,91.0,90.0,1.0,0.0
9,16420,91.0,90.0,1.0,0.0


# Metric

\begin{equation*}
P@K\left(target, predict\right) = \sum_{k=1}^K  r^{true}(\pi^{-1}(k)) + \color{blue} {C_{\beta, L}(\pi^{-1}(k))}
\end{equation*}

\begin{equation*}
C_{\beta, L}(\color{red} p) = \max_{t \in \text{target products}} \sum_{i=0}^L c_{i}^{true}( \color{red} p, t) \cdot \beta^{-i}
\end{equation*}

\begin{equation*}
c_{i}^{true} \left( \color{red} p,  t \right) = \left\{
    \begin{array}{ll}
      1 & \mbox{if predicted product "p" has
      the same category of level "i" as the target product "t"};\\
      0 & \mbox{otherwise}.
    \end{array}
  \right.
  \end{equation*}

In [152]:
def is_relevant(target_products, product):
    return 1.0 if (target_products.product_id == product.product_id).any() > 0 else 0.0

def caregory_score(target_products, product, L=4, beta=2.0):
    max_score = 0.0
    for target_product in target_products:
        score = 0.0
        for level in range(L):
            category = f'category_{level}'
            score += int(product[category] == target_product[category]) * (beta ** (-level))
        if score > max_score:
            max_score = score
            
def p_at_k_modified(target_products, predicted_products, k=50, L=4, beta=2.0):
    print(predicted_products)
    relevant = 0.0
    relevance = predicted_products.apply(lambda p: is_relevant(target_products, p))
    category_scores = predicted_products.apply(lambda p: category_score(target_products, p, L, beta))
    return (relevance + category_scores).sum() / k
    
def ap_at_k_modified(target_products, predicted_products, k=50, L=4, beta=2.0):
    score = 0.0
    for k_ in range(k):
        score += is_relevant(target_products, predicted_products.iloc[k_]) * p_at_k_modified(target_products, predicted_products.loc[:k_ + 1], k_, L, beta) 
    return score / min(k, len(target))

In [153]:
target = products_e.loc[products_e.product_id.isin([54728, 64119])]
predicted = products_e.loc[products_e.product_id.isin([64119, 63057, 100345, 709])]
ap_at_k_modified(target, predicted, k=4, L=L, beta=2.0)

KeyError: 'Cannot get right slice bound for non-unique label: 1'